In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, \
    FloatType, StringType
from pyspark.sql.functions import col, when, count
import csv
import time

# Δημιουργουμε SparkSession με 4 executors
spark = SparkSession \
    .builder \
    .appName("Query 1 with Dataframe") \
    .config("spark.executor.instances", "4") \
    .getOrCreate()

start_time = time.time()

schema = StructType([
    StructField("dr_no", IntegerType(), True),
    StructField("date_rptd", FloatType(), True),
    StructField("date_occ", FloatType(), True),
    StructField("time_occ", StringType(), True),
    StructField("area", StringType(), True),
    StructField("area_name", StringType(), True),
    StructField("rpt_dist_no", StringType(), True),
    StructField("part_1_2", IntegerType(), True),
    StructField("crm_cd", StringType(), True),
    StructField("crm_cd_desc", StringType(), True),
    StructField("mocodes", StringType(), True),
    StructField("vict_age", StringType(), True),
    StructField("vict_sex", StringType(), True),
    StructField("vict_descent", StringType(), True),
    StructField("premis_cd", StringType(), True),
    StructField("premis_desc", StringType(), True),
    StructField("weapon_used_cd", StringType(), True),
    StructField("weapon_desc", StringType(), True),
    StructField("status", StringType(), True),
    StructField("status_desc", StringType(), True),
    StructField("crm_cd_1", StringType(), True),
    StructField("crm_cd_2", StringType(), True),
    StructField("crm_cd_3", StringType(), True),
    StructField("crm_cd_4", StringType(), True),
    StructField("location", StringType(), True),
    StructField("cross_street", StringType(), True),
    StructField("lat", IntegerType(), True),
    StructField("lon", IntegerType(), True),
])

file_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/\
CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
file_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/\
CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

# Διαβαζουμε τα αρχεια και τα ενοποιουμε σε ενα Dataframe
df1 = spark.read.csv(file_1, header=True, schema=schema)
df2 = spark.read.csv(file_2, header=True, schema=schema)
df = df1.union(df2)
# Κραταμε μονο τις στηλες που μας ενδιαφερουν και βγαζουμε τι
# ακυρες περιπτωσεις οπου εχουμε αρνητικη ηλικια θυματων
df = df.filter((col("vict_age") > 0))
df = df.select("vict_age", "crm_cd_desc")
# Φιλτραρουμε ωστε το πεδιο "aggravated assault" να περιέχει ζητουμενο τον όρο
df = df.filter(col("crm_cd_desc").contains("AGGRAVATED ASSAULT"))
# Κατηγοριοποιουμε τα θυματα βασει ηλικιας
df = df.withColumn(
    "age_group",
    when(col("vict_age") < 18, "Kids")
    .when((col("vict_age") >= 18) & (col("vict_age") <= 24), "Young adults")
    .when((col("vict_age") >= 25) & (col("vict_age") <= 64), "Adults")
    .when(col("vict_age") > 64, "Elders")
)
# Μετραμε ποσες φορες εμφανιζεται η εκαστοτε ηλικιακη ομαδα
df = df.groupBy("age_group").agg(count("*").alias("count"))
# Ταξινονομυμε σε φθινουσα σειρα
df = df.orderBy(col("count").desc())
# Εμφανιζουμε τα αποτελεσματα
df.show()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------+
|   age_group| count|
+------------+------+
|      Adults|121093|
|Young adults| 33605|
|        Kids| 10830|
|      Elders|  5985|
+------------+------+

Time taken: 11.81 seconds

In [13]:
sc = SparkSession.builder \
    .appName("Query 1 with RDDs") \
    .config("spark.executor.instances", "4") \
    .getOrCreate() \
    .sparkContext

start_time = time.time()

file_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/\
CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
file_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/\
CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
# Διαβαζουμε τα αρχεια ως RDDs
rdd1 = sc.textFile(file_1)
rdd2 = sc.textFile(file_2)


# Συνaρτηση για ανaλυση CSV
def parse_csv(line):
    return list(csv.reader([line]))[0]


rdd1 = rdd1.map(parse_csv)
rdd2 = rdd2.map(parse_csv)
# Αφαιρουμε την επικεφαλιδα απο καθε RDD
header1 = rdd1.first()
rdd1 = rdd1.filter(lambda row: row != header1)
header2 = rdd2.first()
rdd2 = rdd2.filter(lambda row: row != header2)
# Ενοποιουμε τα δυο RDDs
rdd = rdd1.union(rdd2)
# Δημιουργουμε ενα νεο RDD με τις στηλες που μας ενδιαφερουν
rdd = rdd.map(lambda fields: (fields[11], fields[9]))
# Φιλτραρουμε για ηλικιες μεγαλυτερες από 0
rdd = rdd.filter(lambda x: x[0].isdigit() and int(x[0]) > 0)
# Φιλτραρουμε για "AGGRAVATED ASSAULT"
rdd = rdd.filter(lambda x: "AGGRAVATED ASSAULT" in x[1])


# Κατηγοριοποιουμε θυματα βασει ηλικιας
def categorize_age(age):
    age = int(age)
    if age < 18:
        return "Kids"
    elif 18 <= age <= 24:
        return "Young adults"
    elif 25 <= age <= 64:
        return "Adults"
    else:
        return "Elders"


rdd = rdd.map(lambda x: (categorize_age(x[0]), 1))

# Μετραμε ανα ηλικιακη ομαδα, χρησιμοποιωντας reduceByKey
rdd = rdd.reduceByKey(lambda x, y: x + y)
# Ταξινομουμε σε φθινουσα σειρά
rdd = rdd.sortBy(lambda x: x[1], ascending=False)
# Εμφανιζουμε αποτελεσματα
for result in rdd.collect():
    print(result)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Adults', 121093)
('Young adults', 33605)
('Kids', 10830)
('Elders', 5985)
Time taken: 20.35 seconds